In [ ]:
from enum import Enum
import numpy as np
import pathlib

input_filename = r"programs\fiboancci.mcasm"
output_filename = r"output\fibonacci"

input_filepath = pathlib.Path(input_filename)
output_filepath = pathlib.Path(input_filepath.stem if (output_filename is None) else output_filename)

print(f"Assembling '{input_filepath.resolve()}' -> '{output_filepath.resolve()}'")

### Instruction codes

In [ ]:
class InstructionType(Enum):
    LOAD = 0b0000_0000_0000_0000
    EXEC = 0b1000_0000_0000_0000

class Register(Enum):
    PC = 0
    R1 = 1
    R2 = 2
    R3 = 3
    R4 = 4
    R5 = 5
    R6 = 6
    R7 = 7

class Operation(Enum):
    CLEAR = 0
    FILL = 1
    AND = 2
    NAND = 3
    OR = 4
    NOR = 5
    XOR = 6
    XNOR = 7
    INC = 8
    DEC = 9
    ADD = 10
    SUB = 11
    SHL = 12
    SHR = 13
    ROL = 14
    ROR = 15

class Condition(Enum):
    ALWAYS = 0
    NEVER = 1
    EQUAL = 2
    NOT_EQUAL = 3
    GREATER = 4
    LESS = 5
    GREATER_OR_EQUAL = 6
    LESS_OR_EQUAL = 7

def opcode_load_instruction(relative: bool, condition: Condition, output_register: Register, value: int) -> np.uint16:
    return np.uint16(InstructionType.LOAD.value | ((1 if relative else 0) << 14) | (condition.value << 11) | (output_register.value << 8) | (value & 0xFF))

def opcode_exec_instruction(input_a_reg: Register, input_b_reg: Register, output_reg: Register, operation: Operation) -> np.uint16:
    return np.uint16(InstructionType.EXEC.value | operation.value | (input_a_reg.value << 6) | (input_b_reg.value << 9) | (output_reg.value << 12))

## Load the instruction lines

### Read the src lines

Load all lines from the specified input file

In [ ]:
# Read input lines
with open(input_filepath) as input_file:
    src_lines = input_file.readlines()
src_lines = [line.strip() for line in src_lines]
print(f"Loaded {len(src_lines)} lines from source file:")
print("\n".join(src_lines))

### Get the code lines
This removes all comments, empty lines and multiple whitespace.
This still contains labels

In [ ]:
# Remove comments
code_lines = np.array([line.split('#', 1)[0].strip() for line in src_lines])

# Remove empty lines
src_to_code_line = np.roll(np.cumsum(code_lines != ""), 1)
src_to_code_line[0] = 0
code_line_mapping: list[int] = np.zeros(np.max(src_to_code_line) + 1).tolist()
for i, line in enumerate(src_to_code_line):
    code_line_mapping[line] = i
code_lines = code_lines[code_line_mapping]

# Remove multiple whitespace
code_lines = [" ".join(line.split()) for line in code_lines]

print(f"Loaded {len(code_lines)} lines of code:")
print("\n".join(code_lines))

### Get the instruction lines

This does index all instructions. Additionally all labels are registered and not contained in the resulting list.

In [ ]:
instruction_lines: list[str] = []
instruction_mapping: list[int] = []
labels: dict[str, int] = {}

for i_code_line, line in enumerate(code_lines):
    i_src_line = code_line_mapping[i_code_line]
    if line.startswith("@"):
        label_id = line[1:]
        if label_id in labels:
            raise Exception(f"Trying to declare used label '{label_id}' in line {i_src_line+1}.\nPreviously declared in line {instruction_mapping[labels[label_id]]+1-1}")

        labels[label_id] = len(instruction_lines)
        continue

    instruction_lines.append(line)
    instruction_mapping.append(i_src_line)
n_instructions = len(instruction_lines)

# Replace used labels
instructions_text = "\n".join(instruction_lines)
for label, instruction_id in labels.items():
    instructions_text = instructions_text.replace(f"@{label}", f"{instruction_id}")
instruction_lines = instructions_text.split("\n")

print(f"Loaded {n_instructions} instructions:")
print("\n".join([f"{i:03d}: {line}" for i, line in enumerate(instruction_lines)]))

In [ ]:
print(f"Defined {len(labels)} labels:")
print("\n".join([f"'{id}': instruction {line:03d} in line {instruction_mapping[line]}" for id, line in labels.items()]))

## Parse the instructions

In [ ]:
def parse_register(text: str) -> bool:
    if len(text) != 2:
        return False
    
    text = text.upper()
    if text == "PC":
        return Register.PC
    if text[0] == "R" and text[1].isdigit() and (int(text[1]) in range(8)):
        return list(Register)[int(text[1])]
    return None
    
def is_register_id(text: str) -> Register | None:
    if len(text) != 2:
        return None
    
    text = text.upper()
    if text == "PC":
        return True
    if text[0] == "R" and text[1].isdigit() and (int(text[1]) in range(8)):
        return True
    return False

def int_or_none(text: str) -> int | None:
    try:
        return int(text, 0)
    except ValueError:
        return None

In [ ]:
instructions = np.zeros(n_instructions, dtype=np.uint16)
for i_instruction, instruction_text in enumerate(instruction_lines):
    i_src_line = instruction_mapping[i_instruction]
    src_line = src_lines[i_src_line]
    instruction_text = instruction_text.lower()
    instruction_parts = instruction_text.split(" ")
    n_instruction_parts = len(instruction_parts)
    try:

        # NOP
        if instruction_text == 'nop':
            instructions[i_instruction] = opcode_exec_instruction(Register.R1, Register.R1, Register.R1, Operation.AND)
            continue

        # Assignments
        if n_instruction_parts >= 2 and is_register_id(instruction_parts[0]) and instruction_parts[1] == "=":
            output_register = parse_register(instruction_parts[0])
            if n_instruction_parts == 3:
                if is_register_id(instruction_parts[2]):
                    input_register = parse_register(instruction_parts[2])
                    instructions[i_instruction] = opcode_exec_instruction(input_register, input_register, output_register, Operation.AND)
                    continue

                value = int_or_none(instruction_parts[2])
                if value is not None:
                    instructions[i_instruction] = opcode_load_instruction(False, Condition.ALWAYS, output_register, value)
                    continue

                raise Exception(f"Failed to parse line {i_src_line}: Invalid syntax for load value: '{instruction_parts[2]}'")

            raise Exception(f"Failed to parse line {i_src_line}: Invalid syntax for load: '{src_line}'")

        # CLEAR
        if instruction_parts[0] == "clear":
            if len(instruction_parts) != 2:
                raise Exception(f"Failed to parse line {i_src_line}: Invalid arguments, syntax: 'clear R[0:8]|PC'")
            
            output_register = parse_register(instruction_parts[1])
            if output_register == None:
                raise Exception(f"Failed to parse line {i_src_line}: Invalid register '{instruction_parts[1]}'")
            
            instructions[i_instruction] = opcode_exec_instruction(Register.R1, Register.R1, output_register, Operation.CLEAR)
            continue
    except Exception as exception:
        print(f"Exception whilst parsing line {i_src_line}: '{src_line}'")
        raise exception

    print(f"Failed to parse line {i_src_line}: '{src_line}'")